# FLOWER CLASSIFICATION

In [1]:
import tensorflow as tf
import numpy as num
import os
import glob
import matplotlib.pyplot as plt
import cv2

In [2]:
import keras

In [3]:
training_path=r"D:\Flower_classification\Training_data"
validation_path=r"D:\Flower_classification\Validation_data"
test_path=r"D:\Flower_classification\Test_data"

In [4]:
labels=[]
for i in os.listdir(training_path):
    labels.append(i)
print(labels)  

['Allamanda', 'Anthurium', 'Azalea', 'Balsam', 'Begonia', 'Bellflower', 'Bergenia', 'Bletilla', 'Blue-Star', 'Bougainvillea', 'Brahmakamal', 'Butterflypeal', 'carnation', 'Chrysanthemum', 'Cockscomb', 'Cornflower', 'Cosmos', 'Cuckoo Flower', 'Daffodil', 'daisy', 'Dandelion', 'datura', 'Dhalias', 'Foxglove', 'Frangipani', 'Gerbera', 'gladiolus', 'Gloxinia', 'Hibiscus', 'Hollyhock', 'Iris', 'Ixora', 'jasmine', 'Lavender', 'Lilac', 'Lotus', 'Magnolia', 'Marigold', 'Mirabilis Jalapa', 'Orchid', 'Pansy', 'Periwinkle', 'Petunia', 'Plumeria', 'poppy', 'red vanda', 'Rose', 'Saffron', 'Sunflower', 'Tibouchina', 'tulip', 'Viola', 'Water Lily', 'YellowArchangel', 'Zinnia']


In [5]:
labels[43]

'Plumeria'

# Training Data Generation and Loading in pickle

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rescale=1/255,dtype=num.float16).flow_from_directory(training_path,target_size=(224,224),classes=labels,batch_size=25)

Found 41050 images belonging to 55 classes.


In [7]:
x,y=train_data.next()

# Prepare and Load Validation Data 

In [15]:
val_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rescale=1/255,dtype=num.float16).flow_from_directory(validation_path,target_size=(224,224),classes=labels,batch_size=25)

Found 6416 images belonging to 55 classes.


# Prepare and Load Testing Data 

In [23]:
test_data=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,rescale=1/255,dtype=num.float16).flow_from_directory(test_path,target_size=(224,224),classes=labels,batch_size=25)

Found 6436 images belonging to 55 classes.


In [31]:
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [32]:
mobile=tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

In [33]:
mobile.layers[-7]

In [34]:
custom_layers=mobile.layers[-7].output
add_layer=Flatten()(custom_layers)
output_layer=Dense(55,activation="softmax")(add_layer)
mobile_net_model=Model(inputs=mobile.input,outputs=output_layer)

In [35]:
mobile_net_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [36]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [37]:
for layer in mobile_net_model.layers[:-23]:
    layer.trainable=False

In [38]:
mobile_net_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [38]:
with tf.device('/gpu:0'):
    mobile_net_model.fit(train_data,epochs=100,validation_data=val_data,batch_size=32)

Epoch 1/100
1642/1642 [==============================] - 1283s 782ms/step - loss: 104.2004 - accuracy: 0.6019 - val_loss: 159.2006 - val_accuracy: 0.6256
Epoch 2/100
1642/1642 [==============================] - 710s 432ms/step - loss: 67.5176 - accuracy: 0.7831 - val_loss: 222.1460 - val_accuracy: 0.6127
Epoch 3/100
1642/1642 [==============================] - 260s 158ms/step - loss: 52.8172 - accuracy: 0.8483 - val_loss: 238.6230 - val_accuracy: 0.6747
Epoch 4/100
1642/1642 [==============================] - 192s 117ms/step - loss: 39.9616 - accuracy: 0.8906 - val_loss: 256.1499 - val_accuracy: 0.6997
Epoch 5/100
1642/1642 [==============================] - 192s 117ms/step - loss: 35.6855 - accuracy: 0.9133 - val_loss: 278.0974 - val_accuracy: 0.7138
Epoch 6/100
1642/1642 [==============================] - 192s 117ms/step - loss: 30.2979 - accuracy: 0.9310 - val_loss: 310.2481 - val_accuracy: 0.7241
Epoch 7/100
1642/1642 [==============================] - 194s 118ms/step - loss: 28.90

In [43]:
mobile_net_model.evaluate(val_data)

257/257 [==============================] - 20s 77ms/step - loss: 306.5734 - accuracy: 0.7885


[306.57342529296875, 0.7884975075721741]

In [39]:
for layer in mobile_net_model.layers[:-23]:
    layer.trainable=True

In [40]:
mobile_net_model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [42]:
with tf.device('/gpu:0'):
    mobile_net_model.fit(train_data,epochs=50,validation_data=val_data,batch_size=32)

Epoch 1/50
1642/1642 [==============================] - 983s 595ms/step - loss: 223.1767 - accuracy: 0.8504 - val_loss: 782.1721 - val_accuracy: 0.6488
Epoch 2/50
1642/1642 [==============================] - 548s 333ms/step - loss: 95.5116 - accuracy: 0.8976 - val_loss: 552.1611 - val_accuracy: 0.6828
Epoch 3/50
1642/1642 [==============================] - 528s 321ms/step - loss: 73.7036 - accuracy: 0.9084 - val_loss: 428.6557 - val_accuracy: 0.7118
Epoch 4/50
1642/1642 [==============================] - 525s 320ms/step - loss: 59.4806 - accuracy: 0.9154 - val_loss: 405.0946 - val_accuracy: 0.7204
Epoch 5/50
1642/1642 [==============================] - 526s 320ms/step - loss: 47.4704 - accuracy: 0.9261 - val_loss: 365.9600 - val_accuracy: 0.7352
Epoch 6/50
1642/1642 [==============================] - 528s 321ms/step - loss: 43.3393 - accuracy: 0.9293 - val_loss: 340.2589 - val_accuracy: 0.7463
Epoch 7/50
1642/1642 [==============================] - 529s 322ms/step - loss: 37.7391 - acc

In [44]:
with tf.device('/gpu:0'):
    mobile_net_model.evaluate(test_data)

258/258 [==============================] - 93s 361ms/step - loss: 376.9913 - accuracy: 0.7575


In [45]:
mobile_net_model.save("55_Flowers_mobile_netv1_model.h5")

In [47]:
mobile_netv1_model=keras.models.load_model("55_Flowers_mobile_netv1_model.h5")

In [51]:
weights=mobile_netv1_model.get_weights()

# MOBILE_NET_V2(RES-NET )

In [41]:
mobile_net_v2=tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

In [42]:
mobile_net_v2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [43]:
mobile_net_v2.layers
print(len(mobile_net_v2.layers))

156


# Customize Model

In [44]:
outlayers=mobile_net_v2.layers[-2].output
flat_layer=Flatten()(outlayers)
pred_layer=Dense(55,activation="softmax")(flat_layer)
mobi_V2_model=Model(inputs=mobile_net_v2.input,outputs=pred_layer)

In [45]:
mobi_V2_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [46]:
for layer in mobi_V2_model.layers[:-25]:
    layer.trainable=False

In [47]:
mobi_V2_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [48]:
with tf.device('/gpu:0'):
    mobi_V2_model.fit(train_data,epochs=100,validation_data=val_data,batch_size=32)

Epoch 1/100
1642/1642 [==============================] - 1203s 725ms/step - loss: 0.8665 - accuracy: 0.7603 - val_loss: 2.5069 - val_accuracy: 0.6153
Epoch 2/100
1642/1642 [==============================] - 377s 229ms/step - loss: 0.3332 - accuracy: 0.8981 - val_loss: 3.4462 - val_accuracy: 0.5452
Epoch 3/100
1642/1642 [==============================] - 195s 119ms/step - loss: 0.2090 - accuracy: 0.9344 - val_loss: 2.8350 - val_accuracy: 0.6460
Epoch 4/100
1642/1642 [==============================] - 195s 119ms/step - loss: 0.1599 - accuracy: 0.9492 - val_loss: 2.2453 - val_accuracy: 0.7173
Epoch 5/100
1642/1642 [==============================] - 195s 119ms/step - loss: 0.1224 - accuracy: 0.9607 - val_loss: 1.7676 - val_accuracy: 0.7408
Epoch 6/100
1642/1642 [==============================] - 195s 119ms/step - loss: 0.1045 - accuracy: 0.9659 - val_loss: 2.1260 - val_accuracy: 0.7170
Epoch 7/100
1642/1642 [==============================] - 195s 119ms/step - loss: 0.0913 - accuracy: 0.970

In [49]:
with tf.device('/gpu:0'):
    mobi_V2_model.evaluate(test_data)

258/258 [==============================] - 103s 401ms/step - loss: 2.4143 - accuracy: 0.7913


In [50]:
clone_model=tf.keras.models.clone_model(
    mobi_V2_model
)

In [51]:
clone_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [53]:
clone_weight=clone_model.get_weights()

In [60]:
with tf.device('/gpu:0'):    
    mobi_V2_model.evaluate(val_data)

257/257 [==============================] - 34s 131ms/step - loss: 2.1146 - accuracy: 0.8089


VALIDATION ACCURACY BY MOBI-NETV2 MODEL:80.89%

In [62]:
#save the model
mobi_V2_model.save("55_Flower_classifcation_model_mobinetv2.h5")

D:\movies\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [63]:
load_saved_v2_model=keras.models.load_model("55_Flower_classifcation_model_mobinetv2.h5")

Checking the clone model

In [64]:
with tf.device('/gpu:0'):  
    load_saved_v2_model.evaluate(val_data)

257/257 [==============================] - 29s 110ms/step - loss: 2.1146 - accuracy: 0.8089


In [54]:
for layer in clone_model.layers[:-25]:
    layer.trainable=True

In [55]:
clone_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

# EVALUATING THE TEST SET

In [59]:
with tf.device('/gpu:0'):    
    mobi_V2_model.evaluate(test_data)

258/258 [==============================] - 24s 94ms/step - loss: 2.3104 - accuracy: 0.7920


TEST SET ACCURACY FROM THE MODEL MOBI-NET-V2:79.20

# Feature Extraction from Mobi-Net V1 Model

In [24]:
feature_extract=Model(inputs=mobile_net_model.input,outputs=add_layer)
with tf.device('/gpu:0'):
    extract=feature_extract.predict(train_data)

In [28]:
featre_dump=open("Feature_extracted.pickle","wb")
pickle.dump(extract,featre_dump)
featre_dump.close()

In [82]:
RF=RandomForestClassifier(n_estimators=70,criterion="entropy",random_state=42)

In [46]:
train_data.__len__()

1642

# Fetching the labels

In [73]:
test_labels = []

for i in range(0,1642):
    test_labels.extend(num.array(train_data[i][1]))

In [77]:
len(test_labels)

41050

In [78]:
import pickle
ext=open("Feature_extracted.pickle","rb")
features=pickle.load(ext)

In [88]:
features.shape

(41050, 50176)

In [85]:
test_labels=num.asarray(test_labels,dtype="float32")

In [86]:
test_labels.shape

(41050, 55)